In [46]:
import json
import pickle
import sqlite3
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Kkma
from konlpy.tag import Okt
from konlpy.utils import pprint

In [2]:
con = sqlite3.connect("db.sqlite3")
cur = con.cursor()
# read table 
df_spot = pd.read_sql_query("SELECT * from api_spot", con)

# clonse DB connection
con.close()

In [3]:
df_spot.head()

,id,spot_name,spot_type,address,latitude,longitude,description,overview,image_url
0,125266,국립 청태산자연휴양림,자연관광지,강원 횡성군 둔내면 삽교리 산 1-4,37.522514,128.291912,유모차 대여정보 : 불가<br>신용카드가능 정보 : 없음<br>애완동물 동반가능 정...,"해발 1,200m의 청태산을 주봉으로 하여 인공림과 천연림이 잘 조화된 울창한 산림...",http://tong.visitkorea.or.kr/cms/resource/65/6...
1,125405,토함산자연휴양림,자연관광지,경북 경주시 양북면 장항리 산 599-1,35.757328,129.370895,유모차 대여정보 : 불가<br>신용카드가능 정보 : 가능<br>애완동물 동반가능 정...,<b>※ 코로나바이러스감염증-19 공지사항<br>※ 내용 : 임시휴관 (2020.3...,http://tong.visitkorea.or.kr/cms/resource/40/2...
2,125406,비슬산자연휴양림,자연관광지,대구광역시 달성군 유가면 휴양림길 230(유가면),35.691380,128.515977,유모차 대여정보 : 없음<br>신용카드가능 정보 : 불가<br>애완동물 동반가능 정...,"* 자연경관 안에서 얻는 휴식, 비슬산자연휴양림 *<br />\n비슬산 자연휴양림은...",http://tong.visitkorea.or.kr/cms/resource/62/2...
3,125407,불정자연휴양림,자연관광지,경상북도 문경시 불정길 180(불정동),36.618826,128.134266,유모차 대여정보 : 없음<br>신용카드가능 정보 : 없음<br>애완동물 동반가능 정...,"* 레포츠와 휴식을 동시에, 불정자연휴양림 *<br> \n불정자연휴양림은 경북 문경...",http://tong.visitkorea.or.kr/cms/resource/83/1...
4,125408,청송 자연휴양림 퇴적층 (청송 국가지질공원),자연관광지,경상북도 청송군 부남면 청송로 3478-96(부남면),36.317919,129.053721,유모차 대여정보 : 없음<br>신용카드가능 정보 : 없음<br>애완동물 동반가능 정...,청송자연휴양림 퇴적암층은 상쾌한 공기와 수려한 경관을 모두 갖춘 지질명소이다. 다양...,http://tong.visitkorea.or.kr/cms/resource/48/2...


In [4]:
pd.options.mode.chained_assignment = None
spots = df_spot[['id', 'overview']]
spots.rename(columns={'id':'spot_id'}, inplace=True)

In [5]:
spots.head()

,spot_id,overview
0,125266,"해발 1,200m의 청태산을 주봉으로 하여 인공림과 천연림이 잘 조화된 울창한 산림..."
1,125405,<b>※ 코로나바이러스감염증-19 공지사항<br>※ 내용 : 임시휴관 (2020.3...
2,125406,"* 자연경관 안에서 얻는 휴식, 비슬산자연휴양림 *<br />\n비슬산 자연휴양림은..."
3,125407,"* 레포츠와 휴식을 동시에, 불정자연휴양림 *<br> \n불정자연휴양림은 경북 문경..."
4,125408,청송자연휴양림 퇴적암층은 상쾌한 공기와 수려한 경관을 모두 갖춘 지질명소이다. 다양...


In [12]:
okt = Okt()

In [ ]:
okt.nouns(spots['overview'][0])

In [31]:
nouns = []
for _, spot in spots.iterrows():
    temp = okt.nouns(spot['overview'])
    words = ''
    for t in temp:
        words += t + ' '
    nouns.append([spot['spot_id'], words[:-1]])

In [32]:
nouns_frame = pd.DataFrame(nouns, columns=["spot_id", "description"])

In [35]:
nouns_frame.head()

,spot_id,description
0,125266,해발 의 청 태산 봉 인공 림 림 조화 울창 산림 바탕 국 유림 경영 시범 단지 로...
1,125405,코로나바이러스 감염증 공지 사항 내용 임시 관 미정 여행 정보 변동 사항 페이지 이...
2,125406,자연 경관 안 휴식 비슬산 자연휴양림 비슬산 자연휴양림 대견봉 중심 좌우 조화 봉 ...
3,125407,레포츠 휴식 동시 불정 자연휴양림 불정 자연휴양림 경북 문경 불정동 마을 안쪽 위치...
4,125408,청송 자연휴양림 퇴적암 층 공기 수려 경관 모두 지질 명소 코스 산책로 개설 탐방 ...


In [39]:
nouns_frame.to_pickle('./spot.pkl')

In [6]:
okt_frame = pd.read_pickle('./spot.pkl')

In [7]:
okt_frame.dtypes

spot_id         int64
description    object
dtype: object

In [8]:
tf = TfidfVectorizer(min_df=0)
tfidf_matrix = tf.fit_transform(okt_frame['description'])

In [9]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [39]:
results = {}
for idx, row in okt_frame.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
   similar_items = [(cosine_similarities[idx][i], okt_frame['spot_id'][i]) for i in similar_indices] 
   results[row['spot_id']] = similar_items[1:]

In [98]:
okt_frame.loc[okt_frame['spot_id'] == 125266]['spot_id'][0]

125266

In [34]:
type(results['125266'])

list

In [47]:
with open('spot_result.pickle', 'wb') as f:
    pickle.dump(results, f)

In [48]:
with open('spot_result.pickle', 'rb') as f:
    results = pickle.load(f)

In [50]:
def item(id):
    return okt_frame.loc[okt_frame['spot_id'] == id]['spot_id'].to_string().split()[1]

# Just reads the results out of the dictionary.
def recommend(spot_id, num):
    print("Recommending " + str(num) + " products similar to " + item(spot_id) + "...")   
    print("-------")    
    recs = results[spot_id][:num]
    rec_list = []
    for rec in recs: 
        print("Recommended: " + item(rec[1]) + " (score:" +      str(rec[0]) + ")")
        rec_list.append(item(rec[1]))
    return rec_list

In [51]:
okt_frame.loc[okt_frame['spot_id'] == 125266]['spot_id'].to_string().split()[1]


'125266'

In [52]:
rec_list = recommend(spot_id=125266, num=5)

Recommending 5 products similar to 125266...
-------
Recommended: 127477 (score:0.21025359498473606)
Recommended: 126773 (score:0.2016832488828783)
Recommended: 337517 (score:0.1972319257223753)
Recommended: 2436209 (score:0.1842840865635572)
Recommended: 127821 (score:0.17815608572636613)


In [53]:
for rec in rec_list:
    print(' '.join(df_spot.loc[df_spot['id'] == int(rec)]['spot_name'].to_string().split()[1:]))

군위 장곡자연휴양림
주천강자연휴양림
국립횡성숲체원
안동호반자연휴양림
금강자연휴양림(금강수목원,산림박물관)
